## Тестовое задание MA

Добрый день! В этом ноутбуке содержится моё тестовое задание в компанию MA.
- Трейнинг и оценка модели выполнены в фреймворке TensorFlow / Keras.
- Модель, которая была натренирована на данных, - это Xception (https://arxiv.org/abs/1610.02357). Причина моего выбора лежит в том, что изображения, на которых мне предстояло тренировать модель, были немного шумными, с достаточно небольшой яркостью, временами - некоторым перекосом. В связи с этим, я решил выбрать более "глубокую" структуру Xception, так как основной особенностью этой модели (и предшествующей ей Inception v3) является разветвлённая архитектура, позволяющая модели проводить конволюцию и изучать свойства изображений на разных каналах. Эта модель является более глубокой и сложной, с бо́льшим количеством параметров - в целом это хороший выбор для сложных изображений и, хотя в данной ситуации я мог обойтись простой моделью с парой Conv2D-слоёв, я предпочёл использовать более сложную модель с аугментацией данных для борьбы с переобучением.
- Результаты обучения и тестов Вы можете видеть в конце данного ноутбука.
- Для более наглядной оценки я также сделал:
1) Простой пользовательский интерфейс в Gradio, где Вы можете проверить мою модель на реальных данных. Ссылка:
2) PDF-отчёт, где представлены линии обучения и результаты в виде графика (его можно найти в репозитории на GitHub и HuggingFace)


Структура выполнения задачи:

- Импорт датасетов
- Импорт изображений с помощью ImageDataGenerator, использовать аугментацию данных. Убедиться, что тестовые изображения отделены от тренировочных для недопущения утечки (все названия трейн / валид изображений есть в датасетах)
- Определение модели (из кандидатов: Xception, Regular CNN, YOLOv11m, MobileNetv2), при выборе сложной модели добавить Residual Connections. Конечная задача - мультиклассовый лейбелинг (лосс - категорическая кросс-энтропия)
- Тренировка модели, тюнинг параметров (при наличии тюнабельных)
- Тест модели, оценка результатов (acc, precision, recall)
- Публикация на ГитХаб и HuggingFace с созданием UI в Gradio. Требования для UI: возможность напрямую загрузить изображения, нажать кнопку для инференса, поле для выдачи результата
- Мини-отчёт, PDF

## Импорт библиотек и датасетов, обработка

In [86]:
### Для удобства установки библиотек воспользуйтесь этой командой в терминале:
### !pip install pandas tensorflow numpy scipy opencv-python
### Пожалуйста не обращайте внимания на предупреждения - я делаю этот проект на свежей версии Ubuntu без CUDA-драйверов

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import scipy
import cv2
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Activation  
from tensorflow.keras.models import Model
from tensorflow.keras.applications import Xception
from tensorflow.keras.preprocessing import image


2025-02-05 20:46:28.358019: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-05 20:46:28.363274: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-05 20:46:28.415384: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-05 20:46:28.461322: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738781188.507000    5844 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738781188.51

In [32]:
test_dat = pd.read_csv("test.csv").sort_values(by=['img_name'], ascending=True)
check_list = []

In [33]:
### Для проверки на тест-сете
### Функция для инференса на новых изображениях
def check_new(image_path):
    ### Импорт модели
    model = tf.keras.models.load_model('xception_trained.keras')
    check_img = cv2.imread(image_path)
    check_img = cv2.cvtColor(check_img, cv2.COLOR_BGR2RGB)  
    check_img2 = cv2.resize(check_img, (71, 71)) / 255.0
    check_img2 = np.expand_dims(check_img2, axis=0)
    pred = model.predict(check_img2)
    pred_indices = np.argmax(pred, axis=-1)[0]  
    digits = []
    for idx in pred_indices:
        if idx == 10:
            break
        digits.append(str(idx))
    if digits:
        final_number = "".join(digits)  
    else: 
        final_number = ""
    #save_path = f'results/{str(final_number)}.jpeg'  
    #cv2.imwrite(save_path, check_img) 
    return(final_number)

In [ ]:
for num in test_dat['img_name']:
    f1 = check_new("imgs/" + num)
    f2 = str(num + "  -  " + f1)
    check_list.append(f2)

In [35]:
### Список, в котором зафиксированы все результаты модели
check_list

['481896362_2.jpg  -  499',
 '481896530_2.jpg  -  289',
 '481896607_2.jpg  -  219',
 '481896633_2.jpg  -  1099',
 '481896722_2.jpg  -  799',
 '481896746_2.jpg  -  899',
 '481896751_2.jpg  -  189',
 '481896850_2.jpg  -  649',
 '481896944_2.jpg  -  109',
 '481897078_2.jpg  -  549',
 '481897094_2.jpg  -  899',
 '481897095_2.jpg  -  899',
 '481897151_2.jpg  -  499',
 '483317915_2.jpg  -  949',
 '483319459_2.jpg  -  71',
 '483319541_2.jpg  -  95',
 '483319571_2.jpg  -  159',
 '483319576_2.jpg  -  149',
 '483319625_2.jpg  -  109',
 '483319731_2.jpg  -  139',
 '483319792_2.jpg  -  119',
 '483319825_2.jpg  -  119',
 '483319826_2.jpg  -  189',
 '483319836_2.jpg  -  149',
 '487494940_2.jpg  -  599',
 '487494957_2.jpg  -  659',
 '487494964_2.jpg  -  419',
 '487494977_2.jpg  -  649',
 '487495055_2.jpg  -  2699',
 '487495274_2.jpg  -  859',
 '487495463_2.jpg  -  439',
 '487495495_2.jpg  -  1899',
 '487495522_2.jpg  -  469',
 '487495634_2.jpg  -  469',
 '508748981_2.jpg  -  109',
 '508754093_2.jpg  

In [ ]:
### Здесь Вы можете проверить модель на новом изображении. Запустите эту ячейку, вставьте путь до изображения 
### и проверьте полученный вывод. Внутри этой функции прописан импорт последней лучшей модели, пожалуйста
### убедитесь, что модель находится в той же директории, где был этот ноутбук.
path = input("Пожалуйста вставьте путь до изображения: ")
check_new(str(path))

In [2]:
train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("val.csv")

In [3]:
### Здесь я создам плейсхолдер для тренировки, чтобы затем на эти списки прикрепить тензоры изображений и их ярлыки.
img_train = []
lab_train = []

In [4]:
### Загружаю изображения, заранее меняя их размер на 71х71, т.к. это самый "съедобный" формат для Xception
for index, row in train_df.iterrows():
  name = row['img_name']
  text = row['text']
  img = cv2.imread("imgs/" + name)
  img2 = cv2.resize(img, (71, 71)) / 255.0
  label = int(text)
  digits = [int(d) for d in str(label)]
  img_train.append(img2)
  lab_train.append(digits)

In [8]:
### Этот плейсхолдер также для тренировки, но уже для аугментированных изображений из трейн-датасета. 
### Поскольку Xception - это модель со сложной архитектурой, без аугментации данных мы бы быстро переобучились
img_train_aug = []
lab_train_aug = []

In [15]:
### Для более удобной аугментации я создам функцию, которая принимает два пустых списка и два Series-объекта,
### в которых уже заготовлены тензоры изображения и ярлыки. Полученные результаты аппендятся на списки
### с окончанием "aug". 
def augment(images_to_augment, labels_to_augment, list_img, list_lab):
    datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest') ### Ряд аугментаций, которые будут применяться, среди которых урезка, зум, флип и др.

    for i, (image, label) in enumerate(zip(images_to_augment, labels_to_augment)):
        ### Применяем аугментацию. Для самопроверки пришлось несколько раз делать ресайз.
        augmented_image = datagen.random_transform(image)  # Augment the image
        augmented_image = augmented_image * 255.0  # Scale the values back
        augmented_image = np.clip(augmented_image, 0, 255).astype('uint8')  # Clip values to 0-255 range
        
        ### Для проверки, что аугментация правильно работает
        #save_path = f'aug/aug_{i}.jpeg'  
        #cv2.imwrite(save_path, augmented_image)  
        
        ### Аппенд на пустые списки
        img_normalized = augmented_image.astype('float32') / 255.0 
        list_img.append(img_normalized)
        list_lab.append(label)

In [16]:
augment(img_train, lab_train, img_train_aug, lab_train_aug) ### Применяем функцию на трейн сете

In [32]:
### Получаем полноценные, аугментированные трейн-изображения и ярлыки
X_train = np.array(np.concatenate([img_train, img_train_aug]), dtype=np.float32) 
max_label_length = max(len(label) for label in lab_train)  ### Нужен паддинг ярлыков, модель принимает только послед. одинакового размера
y_train_n_padded = pad_sequences(lab_train, maxlen=max_label_length, padding='post', value=-1)
y_train_aug_padded = pad_sequences(lab_train_aug, maxlen=max_label_length, padding='post', value=-1) ### Паддинг устанавливаем в конце
y_train = np.concatenate([y_train_n_padded, y_train_aug_padded]) ### Получаем полноценный список ярлыков

In [47]:
### Все те же самые операции проводим на валидационном сете. Нужно чтобы градиентный спуск работал от как можно более
### репрезентативной выборки с разнообразными изображениями ценников.
img_val = []
lab_val = []
img_val_aug = []
lab_val_aug = []

In [48]:
for index, row in val_df.iterrows():
  name = row['img_name']
  text = row['text']
  img = cv2.imread("imgs/" + name)
  img2 = cv2.resize(img, (71, 71)) / 255.0
  label = int(text)
  digits = [int(d) for d in str(label)]
  img_val.append(img2)
  lab_val.append(digits)

In [49]:
augment(img_val, lab_val, img_val_aug, lab_val_aug)

In [53]:
X_val = np.array(np.concatenate([img_val, img_val_aug]), dtype=np.float32) 
max_label_length1 = max(len(label) for label in lab_train)  # Find the longest sequence
y_val_n_padded = pad_sequences(lab_val, maxlen=max_label_length1, padding='post', value=-1)
y_val_aug_padded = pad_sequences(lab_val_aug, maxlen=max_label_length1, padding='post', value=-1)
y_val = np.concatenate([y_val_n_padded, y_val_aug_padded])

In [56]:
### Устанавливаем кол-во классов на 11, так как помимо 10 цифр (0-9) также остаётся нейрон для паддинга.
### Проводим one-hot encoding для ярлыков (т.к. задача заключается в классификации).
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
num_classes = 11 
y_train_onehot = np.array([to_categorical(label, num_classes=num_classes) for label in y_train])
y_val_onehot = np.array([to_categorical(label, num_classes=num_classes) for label in y_val])

In [71]:
### К сожалению, эту часть пришлось отыскать на StackOverflow и адапьировать под мой код, т.к. в моей ситуации ярлыки 
### состояли из 4-х разных векторов, и просто использовать стандартные метрики точности-полноты из Тензорфлоу не получлиось бы.
class fl_precision(Precision):
    def update_state(self, y_true, y_pred, sample_weight=None):

        y_true = tf.reshape(y_true, (-1, y_true.shape[-1]))  
        y_pred = tf.reshape(y_pred, (-1, y_pred.shape[-1]))  
        y_true = tf.argmax(y_true, axis=-1)  
        y_pred = tf.argmax(y_pred, axis=-1)  

        super().update_state(y_true, y_pred, sample_weight)

class fl_recall(Recall):
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.reshape(y_true, (-1, y_true.shape[-1]))  
        y_pred = tf.reshape(y_pred, (-1, y_pred.shape[-1]))  

        y_true = tf.argmax(y_true, axis=-1)  
        y_pred = tf.argmax(y_pred, axis=-1)  

        super().update_state(y_true, y_pred, sample_weight)

In [85]:
### Определяем модель. Помимо Xception, я также попробовал некоторые другие архитектуры - к примеру, MobileNetv2, простую
### CNN-ку и, ради эксперимента, Xception + residual connections. Как я убедился, в итоге Xception всё равно побеждает по 
### val_accuracy и val_loss, поэтому решено остаться на этой модели. Для тренировки использую Keras API


metrics = ['accuracy', fl_precision(), fl_recall()]
### Убираем классификатор и претрейн, чтобы натренировать модель с нуля. Загружаем только бэкбоун модели, определяем 
### форму входных данных (71х71, как было раньше установлено).
base = Xception(weights=None, include_top=False, input_shape=(71, 71, 3)) 

### Используем пулинг для того, чтобы уменьшить фича-мапы и оставить только самые нужные активации
### Так как каждый ярлык состоит из 4-х one-hot векторов, все размером 11, нам нужен аутпут в 44 нейрона.
x = GlobalAveragePooling2D()(base.output)  # Output shape: (batch_size, 2048)
x = Dense(4 * 11)(x)
x = Reshape((4, 11))(x)
outputs = Activation('softmax')(x) ### Функция активации выходных данных - Софтмакс, т.к. мы рассматриваем распределение вероятностей.
model = Model(inputs=base.input, outputs=outputs) ### Аутпут в форме логитов
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics) ### Лосс - категорическая кросс-энтропия

### Возможно было использовать EarlyStopping со стратегией на валидационную точность, но т.к. я трейнил модели несколько раз, то 
### эмпирическим путём я убедился, что после 5 эпох точность не повышается, либо же модель начинает оверфиттить. 
### Из-за последнего я также слегка повысил бэтч сайз, чтобы обновления параметров модели не были слишком тщательными.
model.fit(X_train, y_train_onehot, epochs=5, batch_size=64, validation_data=(X_val, y_val_onehot)) 

Epoch 1/5
155/155 ━━━━━━━━━━━━━━━━━━━━ 166s 1s/step - accuracy: 0.5906 - fl_precision_2: 0.9848 - fl_recall_2: 0.9894 - loss: 1.2088 - val_accuracy: 0.4280 - val_fl_precision_2: 0.9808 - val_fl_recall_2: 1.0000 - val_loss: 2.0986
Epoch 2/5
155/155 ━━━━━━━━━━━━━━━━━━━━ 152s 979ms/step - accuracy: 0.8600 - fl_precision_2: 0.9950 - fl_recall_2: 0.9949 - loss: 0.4333 - val_accuracy: 0.5185 - val_fl_precision_2: 0.9808 - val_fl_recall_2: 1.0000 - val_loss: 1.7661
Epoch 3/5
155/155 ━━━━━━━━━━━━━━━━━━━━ 155s 1s/step - accuracy: 0.9233 - fl_precision_2: 0.9974 - fl_recall_2: 0.9982 - loss: 0.2335 - val_accuracy: 0.7423 - val_fl_precision_2: 0.9885 - val_fl_recall_2: 1.0000 - val_loss: 0.8710
Epoch 4/5
155/155 ━━━━━━━━━━━━━━━━━━━━ 161s 1s/step - accuracy: 0.9522 - fl_precision_2: 0.9985 - fl_recall_2: 0.9980 - loss: 0.1438 - val_accuracy: 0.9215 - val_fl_precision_2: 0.9986 - val_fl_recall_2: 0.9963 - val_loss: 0.2749
Epoch 5/5
155/155 ━━━━━━━━━━━━━━━━━━━━ 163s 1s/step - accuracy: 0.9638 - fl_p

In [65]:
model.save('xception_trained.keras')

In [79]:
### Функция для инференса на новых изображениях
def check_new(image_path):
    ### Импорт модели
    model = tf.keras.models.load_model('models/xception_trained.keras')
    check_img = cv2.imread(image_path)
    check_img = cv2.cvtColor(check_img, cv2.COLOR_BGR2RGB)  
    check_img2 = cv2.resize(check_img, (71, 71)) / 255.0
    check_img2 = np.expand_dims(check_img2, axis=0)
    pred = model.predict(check_img2)
    pred_indices = np.argmax(pred, axis=-1)[0]  
    digits = []
    for idx in pred_indices:
        if idx == 10:
            break
        digits.append(str(idx))
    if digits:
        final_number = "".join(digits)  
    else: 
        final_number = ""
    print(final_number)

In [84]:
### Здесь Вы можете проверить модель на новом изображении. Запустите эту ячейку, вставьте путь до изображения 
### и проверьте полученный вывод. Внутри этой функции прописан импорт последней лучшей модели, пожалуйста
### убедитесь, что модель находится в той же директории, где был этот ноутбук. Спасибо за внимание :)
path = input("Пожалуйста вставьте путь до изображения: ")
check_new(str(path))

Пожалуйста вставьте путь до изображения:  imgs/511270808_2.jpg


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 432ms/step
199
